In [1]:
import cv2
import numpy as np
import os
import pandas as pd
import json
import collections
import string
import re

In [2]:
def load_json_in_dict(path):
    ''' 
    Loads ordered dictionnary of bounds per pages and per coordinates from a json file
    :param string path: where all data of libretto is stored, as explained above
    '''
    with open(path) as json_file: 
        return json.load(json_file)

In [3]:
data = load_json_in_dict("./data/Antigone/3_Network/network.json")

In [146]:
# Create links
network_array_links = pd.DataFrame(columns=['source', 'target'])
network_array_nodes = pd.DataFrame(columns=['id'])
for act in data.keys():
    name= "act"+act
    network_array_links[name] = 0
    network_array_nodes[name] = 0
    for scene_i in data[act].keys():
        characters = sorted(list(data[act][scene_i].keys()))
        for i, character_i in enumerate(characters):
            if not network_array_nodes['id'][network_array_nodes['id'].isin([character_i])].empty:
                network_array_nodes.loc[network_array_nodes['id']==character_i, name] += 1
            else:
                network_array_nodes = network_array_nodes.append({"id":character_i, name:1}, ignore_index=True)

            for character_j in characters[i+1:]:
                if not network_array_links['source'][network_array_links['source'].isin([character_i])].empty:
                    char_i = network_array_links['source']==character_i
                    if not network_array_links[char_i]['target'][network_array_links[char_i]['target'].isin([character_j])].empty:
                        char_j = network_array_links['target']==character_j
                        network_array_links.loc[char_i & char_j, name] += 1
                    else:
                        network_array_links = network_array_links.append({"source":character_i, "target":character_j, name:1}, ignore_index=True)
                else:
                    network_array_links = network_array_links.append({"source":character_i, "target":character_j, name:1}, ignore_index=True)

In [149]:
network = {"nodes": network_array_nodes.to_dict('records'), "links": network_array_links.to_dict('records')}
network

{'nodes': [{'id': 'Creonte', 'act1': 3, 'act2': 6, 'act3': 5},
  {'id': 'Eurifteo', 'act1': 8, 'act2': 8, 'act3': 6},
  {'id': 'Learco', 'act1': 6, 'act2': 6, 'act3': 10},
  {'id': 'Alcefte', 'act1': 3, 'act2': 8, 'act3': 4},
  {'id': 'Antigona', 'act1': 6, 'act2': 6, 'act3': 6},
  {'id': 'Ermione', 'act1': 3, 'act2': 3, 'act3': 6}],
 'links': [{'source': 'Creonte',
   'target': 'Eurifteo',
   'act1': 3,
   'act2': 5,
   'act3': 3},
  {'source': 'Creonte', 'target': 'Learco', 'act1': 3, 'act2': 3, 'act3': 4},
  {'source': 'Eurifteo', 'target': 'Learco', 'act1': 5, 'act2': 3, 'act3': 5},
  {'source': 'Alcefte', 'target': 'Creonte', 'act1': 2, 'act2': 4, 'act3': 2},
  {'source': 'Alcefte', 'target': 'Eurifteo', 'act1': 2, 'act2': 6, 'act3': 2},
  {'source': 'Alcefte', 'target': 'Learco', 'act1': 3, 'act2': 5, 'act3': 4},
  {'source': 'Antigona', 'target': 'Ermione', 'act1': 3, 'act2': 1, 'act3': 4},
  {'source': 'Antigona',
   'target': 'Eurifteo',
   'act1': 4,
   'act2': 4,
   'act3': 

In [150]:
def save_dict_in_json(dictionnary, path):
    ''' Saves ordered dictionnary of bounds per pages and per coordinates in a json file'''
    with open(path, "w") as outfile:  
        json.dump(dictionnary, outfile) 

In [151]:
save_dict_in_json(network, "./data/Antigone/3_Network/network_2.json")